In [1]:
!pip install allennlp==1.0.0 allennlp-models==1.0.0
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [2]:
from Utils.ConfigManager import *

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [3]:
#You can modify and add any text you want to parse

texts = [["You are pedro aznar, a knight living in the kingdom of Larion",
           "You have a steel longsword and a wooden shield",
           "You are on a quest to defeat the evil dragon of Larion",
           r"You've heard he lives up at the north of the kingdom",
           "You set on the path to defeat him and walk into a dark forest",
           "As you enter the forest you see a figure in black",
           "He looks at you and start walking at you"],
          ["You are pedro aznar, a knight living in the kingdom of Larion",
           "You have a steel longsword and a wooden shield",
           "You are on a quest to defeat the evil dragon of Larion",
           r"You've heard he lives up at the north of the kingdom",
           "You set on the path to defeat him and walk into a dark forest",
           "As you enter the forest you see a homunculus wearing a black cloak and a red crown on his head",
           "You draw your steel longsword, preparing for a fight"]
          ]

### Initial config

In [4]:
cm = ConfigManager()

cm.set_model_hiperparams(
    character_name="pedro aznar",
    threshold=0.5,
    nlp_type="en_core_web_sm",
    predictor_path="https://storage.googleapis.com/allennlp-public-models/bert-base-srl-2020.03.24.tar.gz"
)

In [5]:
text_choosen = 0
i = 0

ep = cm.get_entity_parser()
pp = cm.get_possesions_parser()
ap = cm.get_actions_parser()

### Run iteratively

In [10]:
#Run this cell multiple times to get iterative output

if i < len(texts[text_choosen]):
    text = texts[text_choosen][i]

    ep.process(text, True)

    processed_text = str(ep.naive_coref())
    pp.process(processed_text, ep.all_entities_)
    ap.process(processed_text, ep.all_entities_, ep.description_)

    print("text:            ", ep.text_)
    print("entities:        ", ep.all_entities_)
    print("descriptions:    ", ep.description_)
    print("possesions:      ", pp.possesions_)
    print("actions:")
    for act in ap.actions_:
        print(f"    {act}")
    i += 1

else:
    print("Restarting index")
    i = 0
    ep = EntityParser(character_name, use_corefs, debug)
    pp = PossesionsParser(debug=False)
    ap = ActionsParser(debug=True)

text:             You set on the path to defeat him and walk into a dark forest
entities:         {'PERSON_0': 'pedro aznar', 'GPE_0': 'Larion', 'PERSON_1': 'dragon', 'GPE_1': 'north', 'GPE_2': 'forest', 'GPE_3': 'path'}
descriptions:     {'GPE_0': 'kingdom', 'PERSON_0': 'knight'}
possesions:       {'PERSON_0': ['longsword', 'shield']}
actions:
    {pedro aznar: [{'V': 'living'}, {'RECEIVER': 'Deleted'}, {'LOC': Larion}]}
    {pedro aznar: [{'V': 'defeat'}, {'RECEIVER': dragon}, {'LOC': 'Deleted'}]}
    {pedro aznar: [{'V': 'heard'}, {'RECEIVER': dragon}, {'LOC': 'Deleted'}]}
    {dragon: [{'V': 'lives'}, {'RECEIVER': 'Deleted'}, {'LOC': Larion}]}
    {pedro aznar: [{'V': 'set'}, {'RECEIVER': dragon}, {'LOC': 'Deleted'}]}
    {pedro aznar: [{'V': 'defeat'}, {'RECEIVER': dragon}, {'LOC': 'Deleted'}]}
    {pedro aznar: [{'V': 'walk'}, {'RECEIVER': 'Deleted'}, {'LOC': forest}]}


In [12]:
ep.display_processed_text()

### Run all (speed test)

In [14]:
import time
from statistics import mean

ep = cm.get_entity_parser()
pp = cm.get_possesions_parser()
ap = cm.get_actions_parser()

start_for_loop = int(round(time.time() * 1000))
start_ep = []
end_ep = []
start_pp = []
end_pp = []
start_ap = []
end_ap = []

for text in texts[text_choosen]:
    start_ep.append(int(round(time.time() * 1000)))
    ep.process(text, True)
    end_ep.append(int(round(time.time() * 1000)))

    processed_text = str(ep.naive_coref())

    start_pp.append(int(round(time.time() * 1000)))
    pp.process(processed_text, ep.all_entities_)
    end_pp.append(int(round(time.time() * 1000)))

    start_ap.append(int(round(time.time() * 1000)))
    ap.process(processed_text, ep.all_entities_, ep.description_)
    end_ap.append(int(round(time.time() * 1000)))

end_for_loop = int(round(time.time() * 1000))


print("total time elapsed: ", (end_for_loop - start_for_loop) / 1000, "s")
print("time elapsed entities parser:   ", (mean([e - s for s, e in zip(start_ep, end_ep)])) / 1000, "s")
print("time elapsed possesions parser: ", (mean([e - s for s, e in zip(start_pp, end_pp)])) / 1000, "s")
print("time elapsed actions parser:    ", (mean([e - s for s, e in zip(start_ap, end_ap)])) / 1000, "s")
print("entities:       ", ep.all_entities_)
print("descriptions:   ", ep.description_)
print("possesions:     ", pp.possesions_)
print("actions:")
for act in ap.actions_:
    print(f"    {act}")

total time elapsed:  1.778 s
time elapsed entities parser:    0.025142857142857144 s
time elapsed possesions parser:  0.046142857142857145 s
time elapsed actions parser:     0.18271428571428572 s
entities:        {'PERSON_0': 'pedro aznar', 'GPE_0': 'Larion', 'PERSON_1': 'dragon', 'GPE_1': 'north', 'GPE_2': 'path', 'GPE_3': 'forest', 'PERSON_2': 'cloak', 'PERSON_3': 'homunculus', 'GPE_4': 'head', 'PERSON_4': 'longsword'}
descriptions:    {'GPE_0': 'kingdom', 'PERSON_0': 'knight'}
possesions:      {'PERSON_0': ['longsword', 'shield'], 'PERSON_2': ['cloak', 'crown']}
actions:
    {pedro aznar: [{'V': 'living'}, {'RECEIVER': 'Deleted'}, {'LOC': Larion}]}
    {pedro aznar: [{'V': 'defeat'}, {'RECEIVER': dragon}, {'LOC': 'Deleted'}]}
    {pedro aznar: [{'V': 'heard'}, {'RECEIVER': dragon}, {'LOC': 'Deleted'}]}
    {dragon: [{'V': 'lives'}, {'RECEIVER': 'Deleted'}, {'LOC': Larion}]}
    {pedro aznar: [{'V': 'set'}, {'RECEIVER': dragon}, {'LOC': 'Deleted'}]}
    {pedro aznar: [{'V': 'defeat'}